## Решение соревнования https://www.kaggle.com/c/dmia-surnames-classification
##### Нужно построить классификатор, определяющий, является ли слово фамилией. Для оценки качества используется площадь под ROC-кривой (AUC), так как классы сильно несбалансированны.
#### Кощегулов Эльдар Public - 0.87451 Private - 0.87753

In [1]:
import pandas as pd
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn import cross_validation
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.metrics import roc_curve, auc
import xgboost as xgb
import pymorphy2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Users\m1ckyro5aololo\Anaconda2\envs\py36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
label_start = train['Label']
res = pd.DataFrame(test.index, columns = ['Id'])

In [3]:
train['Word'] = train['Word'].apply(str.strip)
test['Word'] = test['Word'].apply(str.strip)

#### Генерим фичи 

In [4]:
train['big'] = train['Word'].apply(lambda x : 1 if list(x)[0].lower() != list(x)[0] else 0)
test['big'] = test['Word'].apply(lambda x : 1 if list(x)[0].lower() != list(x)[0] else 0)

In [5]:
train['upper'] = train['Word'].apply(lambda x : 1 if x.upper() == x else 0)
test['upper'] = test['Word'].apply(lambda x : 1 if x.upper() == x else 0)

In [6]:
train['lower'] = train['Word'].apply(lambda x : 1 if x.lower() == x else 0)
test['lower'] = test['Word'].apply(lambda x : 1 if x.lower() == x else 0)

In [7]:
train['apostrophe'] = train['Word'].apply(lambda x : 1 if x.find("'") != -1 else 0)
train['hyphen'] = train['Word'].apply(lambda x : 1 if x.find("-") != -1 else 0)

In [8]:
test['apostrophe'] = test['Word'].apply(lambda x : 1 if x.find("'") != -1 else 0)
test['hyphen'] = test['Word'].apply(lambda x : 1 if x.find("-") != -1 else 0)

In [9]:
train['Word'] = train['Word'].apply(str.lower)
test['Word'] = test['Word'].apply(str.lower)

#### Боремся с мусором

In [10]:
train['Word'] = train['Word'].str.replace('á', 'а')
train['Word'] = train['Word'].str.replace('é', 'е')
train['Word'] = train['Word'].str.replace('ѐ', 'е')
train['Word'] = train['Word'].str.replace('ё', 'е')
train['Word'] = train['Word'].str.replace('ë', 'е')
train['Word'] = train['Word'].str.replace('ó', 'о')
train['Word'] = train['Word'].str.replace('ô', 'о')
train['Word'] = train['Word'].str.replace('ó', 'о')
train['Word'] = train['Word'].str.replace("'", "'")
train['Word'] = train['Word'].str.replace('’', "'")
train['Word'] = train['Word'].str.replace('”', "'")
train['Word'] = train['Word'].str.replace('“', "'")
train['Word'] = train['Word'].str.replace('`', "'")
train['Word'] = train['Word'].str.replace('"', "'")

In [11]:
test['Word'] = test['Word'].str.replace('á', 'а')
test['Word'] = test['Word'].str.replace('é', 'е')
test['Word'] = test['Word'].str.replace('ѐ', 'е')
test['Word'] = test['Word'].str.replace('ё', 'е')
test['Word'] = test['Word'].str.replace('ë', 'е')
test['Word'] = test['Word'].str.replace('ó', 'о')
test['Word'] = test['Word'].str.replace('ô', 'о')
test['Word'] = test['Word'].str.replace('ó', 'о')
test['Word'] = test['Word'].str.replace("'", "'")
test['Word'] = test['Word'].str.replace('’', "'")
test['Word'] = test['Word'].str.replace('”', "'")
test['Word'] = test['Word'].str.replace('“', "'")
test['Word'] = test['Word'].str.replace('`', "'")
test['Word'] = test['Word'].str.replace('"', "'")

In [12]:
let_remove = ['.', '/', '2', '3', '4', '7', '«', '·', '»', "'", '-', ' ', '&', '0', '1', '\\', '\xa0']
for let in let_remove :
    train['Word'] = train['Word'].str.replace(let, '')  
    test['Word'] = test['Word'].str.replace(let, '')   

#### Генерим читерские фичи и преобразуем слово к нормальной форме, используя библиотеку pymorphy2 

In [13]:
morph = pymorphy2.MorphAnalyzer()

In [14]:
def f(x) :
    #print (x)
    res = morph.parse(x)
    for tmp in res :
        #print (tmp.tag)
        if set(['Surn', 'sing']) in tmp.tag :
            return tmp.inflect({'nomn', 'sing'}).normal_form   
    return res[0].normal_form

In [15]:
def g(x) :
    #print (x)
    res = morph.parse(x)
    for tmp in res :
        #print (tmp.tag)
        if set(['Surn', 'sing']) in tmp.tag :
            return 1   
    return 0

In [16]:
train['pymorphy'] = train['Word'].apply(g)
test['pymorphy'] = test['Word'].apply(g)

In [17]:
train['Word'] = train['Word'].apply(f)
test['Word'] = test['Word'].apply(f)

#### Генерим фичи : длина, количество гласных/согласных, оканчивается на буквы 'xy'/'xyz'

In [18]:
train['len'] = train['Word'].apply(len)
train['glasn'] = train['Word'].apply(lambda x : sum(list(map(x.count, 'аеёиоуэюяы'))))
train['soglasn'] = train['len'] - train['glasn']
train['2let'] = train['Word'].apply(lambda x : ''.join(list(x)[-2 :]))
train['3let'] = train['Word'].apply(lambda x : ''.join(list(x)[-3 :]))

In [19]:
test['len'] = test['Word'].apply(len)
test['glasn'] = test['Word'].apply(lambda x : sum(list(map(x.count, 'аеёиоуэюяы'))))
test['soglasn'] = test['len'] - test['glasn']
test['2let'] = test['Word'].apply(lambda x : ''.join(list(x)[-2 :]))
test['3let'] = test['Word'].apply(lambda x : ''.join(list(x)[-3 :]))

In [20]:
col2 = (train['2let'].value_counts() > train['2let'].value_counts().median()).index

In [21]:
col3 = (train['3let'].value_counts() > train['3let'].value_counts().median()).index

In [22]:
for let2 in col2[: 50] :
    train['last' + '_' + let2] = train['2let'].apply(lambda x : 1 if x == let2 else 0)   
    test['last' + '_' + let2] = test['2let'].apply(lambda x : 1 if x == let2 else 0)   

In [23]:
for let3 in col3[: 50] :
    train['last' + '_' + let3] = train['3let'].apply(lambda x : 1 if x == let3 else 0)   
    test['last' + '_' + let3] = test['3let'].apply(lambda x : 1 if x == let3 else 0)  

#### Генерим фичи частоты окончаний вида 'xy'/'xyz'
<font color = 'red'>Эти 2 фичи добавил уже после окончания соревнования(дают +0.02 к auc)<font>

In [24]:
train['2let_freq'] = train['2let'].map(train['2let'].value_counts() / train.shape[0])
train['3let_freq'] = train['3let'].map(train['3let'].value_counts() / train.shape[0])
test['2let_freq'] = test['2let'].map(train['2let'].value_counts() / train.shape[0])
test['3let_freq'] = test['3let'].map(train['3let'].value_counts() / train.shape[0])
test['2let_freq'] = test['2let_freq'].fillna(0)
test['3let_freq'] = test['3let_freq'].fillna(0)

#### Все это грузим в xgboost

In [25]:
label = train['Label']
train = train.drop(['Word', 'Label', '2let', '3let'], axis = 1)
test = test.drop(['Word', '2let', '3let'], axis = 1)

In [26]:
x_train,  x_valid, y_train, y_valid = train_test_split(train, label, random_state = 456)

print('Building DMatrix...')
d_train = xgb.DMatrix(x_train, label = y_train)
d_valid = xgb.DMatrix(x_valid, label = y_valid)

print('Training ...')

params = {'max_depth': 10, 
          'min_child_weight': 50, 
          'lambda': 0.1,
          'alpha': 0.1,
          'eta': 0.5, 
          'objective': 'binary:logistic',
          'eval_metric' : 'auc',
          'silent': 1,
          'seed': 95
         }

watchlist = [(d_train, 'train'), (d_valid, 'valid')]
model1 = xgb.train(params, d_train, 30000, watchlist, early_stopping_rounds = 100, verbose_eval = 10)
d_train_full = xgb.DMatrix(train, label)
print('AUC XgBoost: ', roc_auc_score(label_start, model1.predict(d_train_full)))

Binding to float32
Building DMatrix...
Training ...
[0]	train-auc:0.883556	valid-auc:0.877586
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 100 rounds.
[10]	train-auc:0.90401	valid-auc:0.891968
[20]	train-auc:0.909948	valid-auc:0.894179
[30]	train-auc:0.914063	valid-auc:0.895756
[40]	train-auc:0.916522	valid-auc:0.896794
[50]	train-auc:0.918742	valid-auc:0.897595
[60]	train-auc:0.921013	valid-auc:0.898273
[70]	train-auc:0.922426	valid-auc:0.898949
[80]	train-auc:0.923833	valid-auc:0.898549
[90]	train-auc:0.925493	valid-auc:0.899082
[100]	train-auc:0.926483	valid-auc:0.899353
[110]	train-auc:0.927669	valid-auc:0.89956
[120]	train-auc:0.928731	valid-auc:0.899487
[130]	train-auc:0.929902	valid-auc:0.899881
[140]	train-auc:0.930911	valid-auc:0.899805
[150]	train-auc:0.931953	valid-auc:0.900086
[160]	train-auc:0.93271	valid-auc:0.900054
[170]	train-auc:0.933681	valid-auc:0.900195
[180]	train-auc:0.934572	v

In [27]:
d_test = xgb.DMatrix(test)
res['Prediction'] = model1.predict(d_test)
res.to_csv('data/res.csv', index = False)